In [14]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env
from pyomo.environ import Constraint

m = wt.watertap_setup(dynamic=False)


wt.case_study_trains.train = {"case_study": "santa_barbara",
                             "reference": "nawi",
                             "scenario": "baseline"}


wt.case_study_trains.source_water = {"case_study": "santa_barbara", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": "seawater"}

m = wt.case_study_trains.get_case_study(m=m)


santa_barbara

------- Adding Unit Processes -------
sw_onshore_intake
ferric_chloride_addition
chlorination
static_mixer
holding_tank
media_filtration
anti_scalant_addition
cartridge_filtration
reverse_osmosis
holding_tank_b
uv_aop
co2_addition
lime_addition
treated_storage
backwash_solids_handling
landfill
municipal_drinking
-------------------------------------

adding source

Connecting unit processes...


In [15]:
wt.display.show_train2(model_name=m)

In [16]:
### # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("Unfixing feed presure and area for", key, '...\n')

wt.run_water_tap(m=m, objective=True, skip_small=True)

wt.run_water_tap(m=m, objective=True, print_model_results="summary", skip_small=True)
wt.print_ro_results(m)
       

Unfixing feed presure and area for reverse_osmosis ...

----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


SW ONSHORE INTAKE:

	total cap investment: 2.00857
	cat and chem cost: 0.0
	electricity cost: 0.13389
	total fixed op cost: 0.02665

FERRIC CHLORIDE ADDITION:

	total cap investment: 1.6475
	cat and chem cost: 0.24895
	electricity cost: 0.0
	total fixed op cost: 0.02194

CHLORINATION:

	total cap investment: 5.20644
	cat and chem cost: 0.0345
	electricity cost: 1e-05
	total fixed op cost: 0.06933

STATIC MIXER:

	total cap investment: 0.11622
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost:

In [17]:
# ONLY FOR DEBUGGING
# Find bounds for RO pessure:
# m.fs.objective_function.deactivate()

# for key in m.fs.pfd_dict.keys():
#     if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        
#         stash_value = value(getattr(m.fs, key).feed.pressure[0])
#         getattr(m.fs, key).feed.pressure.unfix()
#         m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).feed.pressure[0], sense=env.minimize)
#         wt.run_water_tap(m = m, objective=False, skip_small = True)
#         print("LCOW -->", m.fs.costing.LCOW())
#         print(key, "Minimum -->", getattr(m.fs, key).feed.pressure[0]())

#         m.fs.objective_function1 = env.Objective(expr=getattr(m.fs, key).feed.pressure[0], sense=env.maximize)
#         wt.run_water_tap(m = m, objective=False, skip_small = True)
#         print("LCOW -->", m.fs.costing.LCOW())
#         print(key, "Maximum -->", getattr(m.fs, key).feed.pressure[0]())
        
#         getattr(m.fs, key).feed.pressure.unfix()
        
# m.fs.objective_function1.deactivate()

In [18]:
# add more reasonable flux constraints --> THIS CAN AFFECT WATER RECOVERY! MAY NEED TO ADJUST TO NOT OVER CONSTRAIN.
feed_flux_max = 40 #lmh
reject_flux_min = 2.75 #lmh
b = [0.2*1.15, 0.9*0.85]
a = [2.5*1.15, 9*0.85]
min_mem_cost = 25
max_pressure = 84

q=1
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        setattr(m.fs, ("flux_constraint%s" % q), Constraint(
            expr=getattr(m.fs, key).feed.water_flux[0] * 3600 <= feed_flux_max)
               )
        q = q + 1
        
        setattr(m.fs, ("flux_constraint%s" % q), Constraint(
            expr=getattr(m.fs, key).mem_cost[0] >= min_mem_cost)
               )
        q = q + 1
        
        setattr(m, ("flux_constraint%s" % q), Constraint(
            expr=getattr(m.fs, key).retenate.water_flux[0] * 3600 >= reject_flux_min)
               )
        q = q + 1
        
        setattr(m, ("flux_constraint%s" % q), Constraint(
            expr=getattr(m.fs, key).feed.pressure[0] <= max_pressure)
               )
        q = q + 1        
        
        setattr(m.fs, ("flux_constraint%s" % q), Constraint(
            expr=getattr(m.fs, key).a[0] <= a[1])
               )
        q = q + 1
        setattr(m.fs, ("flux_constraint%s" % q), Constraint(
            expr=getattr(m.fs, key).a[0] >= a[0])
               )
        q = q + 1
        setattr(m.fs, ("flux_constraint%s" % q), Constraint(
            expr=getattr(m.fs, key).b[0] <= b[1])
               )
        q = q + 1
        setattr(m.fs, ("flux_constraint%s" % q), Constraint(
            expr=getattr(m.fs, key).b[0] >= b[0])
               )
        q = q + 1
        
wt.run_water_tap(m=m, objective=True, print_model_results="summary", skip_small=True)

wt.print_ro_results(m)

----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


SW ONSHORE INTAKE:

	total cap investment: 2.00857
	cat and chem cost: 0.0
	electricity cost: 0.13389
	total fixed op cost: 0.02665

FERRIC CHLORIDE ADDITION:

	total cap investment: 1.6475
	cat and chem cost: 0.24895
	electricity cost: 0.0
	total fixed op cost: 0.02194

CHLORINATION:

	total cap investment: 5.20644
	cat and chem cost: 0.0345
	electricity cost: 1e-05
	total fixed op cost: 0.06933

STATIC MIXER:

	total cap investment: 0.11622
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00154

HOLDING TANK:

	total cap investment: 1.48059
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.01946

MEDIA FILTRATION:

	total cap investment: 2.68667
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost

In [19]:
# If you need the system recovery to match better.... set a maximum recovery rate.
# The previous system recovery results must be greater than the limit you set below. If this is not the case, then
# you need to ease the bounds on the flux assumpations.

# desired_recovery = 0.44

# if m.fs.costing.system_recovery() < desired_recovery:
#     m.fs.recovery_bound = Constraint(expr=m.fs.costing.system_recovery >= desired_recovery)

#     wt.run_water_tap(m=m, objective=True, print_model_results="summary", skip_small=True)

#     wt.print_ro_results(m)

# else:
#     print("system recovery already lower than desired recovery. desired:", desired_recovery, 
#           "current:", m.fs.costing.system_recovery())

In [20]:
# MAKE SURE THAT CURRENT PRESSURE IS NOT AT LIMITS IF WANTING TO DO SENSITIVITY ANALYSES.
# MAKE SURE THAT AREA IS NOT VERY SMALL OR VERY LARGE IF WANTING TO DO SENSITIVITY ANALYSES.
# MAKE SURE THAT FEED PRESSURES ARE REASONABLE --> ~55-75bar for first pass RO, ~10-30 bar for bracksih first pass
# second pass / stage ??

In [21]:
# store RO variables
ro_stash = {}
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        ro_stash[key] = {"feed.pressure" : getattr(m.fs, key).feed.pressure[0](),
        "membrane_area" : getattr(m.fs, key).membrane_area[0](),
        "a" : getattr(m.fs, key).a[0](),
        "b" : getattr(m.fs, key).b[0]()}

In [22]:
# set everything and deactivate constraints
m = wt.watertap_setup(dynamic=False)
m = wt.case_study_trains.get_case_study(m=m)

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("Unfixing feed presure and area for", key, '...\n')
        
wt.run_water_tap(m=m, objective=True, skip_small=True)

# set variables so that degrees of freedom is zero
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.fix(ro_stash[key]["feed.pressure"])
        getattr(m.fs, key).membrane_area.fix(ro_stash[key]["membrane_area"])
        getattr(m.fs, key).a.fix(ro_stash[key]["a"])
        getattr(m.fs, key).b.fix(ro_stash[key]["b"])

# run model to make sure it still works
#m.fs.objective_function.deactivate()
wt.run_water_tap(m=m, objective=False, print_model_results="summary", skip_small=True)

wt.print_ro_results(m)           

santa_barbara

------- Adding Unit Processes -------
sw_onshore_intake
ferric_chloride_addition
chlorination
static_mixer
holding_tank
media_filtration
anti_scalant_addition
cartridge_filtration
reverse_osmosis
holding_tank_b
uv_aop
co2_addition
lime_addition
treated_storage
backwash_solids_handling
landfill
municipal_drinking
-------------------------------------

adding source

Connecting unit processes...
Unfixing feed presure and area for reverse_osmosis ...

----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


SW ONSHORE INTAKE:

	total cap investment: 2.00857
	cat and chem cost: 0.0
	electricity cost: 0.13389
	

In [62]:
###### SENSITIVITY ANALYSES 

In [11]:
# 
q=1
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).a.unfix()
        getattr(m.fs, key).b.unfix()
        
        b = [getattr(m.fs, key).b[0]()*1.10, getattr(m.fs, key).b[0]()*0.90]
        a = [getattr(m.fs, key).a[0]()*1.10, getattr(m.fs, key).a[0]()*0.90]
        print(b)
        print(a)
        setattr(m.fs, ("flux_constraint%s" % q), Constraint(
            expr=getattr(m.fs, key).a[0] >= a[1])
               )
        q = q + 1
        setattr(m.fs, ("flux_constraint%s" % q), Constraint(
            expr=getattr(m.fs, key).a[0] <= a[0])
               )
        q = q + 1
        setattr(m.fs, ("flux_constraint%s" % q), Constraint(
            expr=getattr(m.fs, key).b[0] >= b[1])
               )
        q = q + 1
        setattr(m.fs, ("flux_constraint%s" % q), Constraint(
            expr=getattr(m.fs, key).b[0] <= b[0])
               )
        q = q + 1
        

[0.4133914934299761, 0.338229403715435]
[3.162500003061112, 2.587500002504546]


In [12]:
# run to rest before sensitivity

# sensitivity
case_study=wt.case_study_trains.source_water["case_study"]
scenario=wt.case_study_trains.source_water["scenario"]
wt.sensitivity_runs(m = m, save_results = True, scenario = scenario, case_study = case_study, skip_small_sens = True)

------- Plant Capacity Utilization 70-100% -------
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.7 LCOW --> 1.8467681710087485
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.715 LCOW --> 1.8080247828771394
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.73 LCOW --> 1.7708735887977785
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.07000000000000002 LCOW --> 1.3997056437243098
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.07250000000000002 LCOW --> 1.4136293929521329
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.07500000000000002 LCOW --> 1.4276698359888929
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Weighted Average Cost of Capital 5-10% 0.077500000000


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-20% 39.2 LCOW --> 1.4066742748861842
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-20% 39.72500000000001 LCOW --> 1.4267952549149872
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet TDS +-20% 40.250000000000014 LCOW --> 1.447716334791849
------- RESET -------
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 1.2927377209611206
------- Inlet Flow +-20% -------
----------------------------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 25.5 LCOW --> 1.2171474648924274
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 27.0 LCOW --> 1.2023975165436862
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 28.5 LCOW --> 1.1894047433666894
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 30.0 LCOW --> 1.177901882957824
----------------------------------------------------------------------

Degrees of 


WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.16213999999999992 LCOW --> 1.3788059689433254
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.16615999999999992 LCOW --> 1.3911014329407794
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.17017999999999991 LCOW --> 1.4033968969382309
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.1741999999999999 LCOW --> 1.4156923609356817
------- RESET -------
-


WaterTAP3 solution optimal 

----------------------------------------------------------------------
pressure 52.024268887778675 LCOW --> 1.4217517006453961
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
pressure 52.629202246938895 LCOW --> 1.4046097553704753
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
pressure 53.234135606099116 LCOW --> 1.388608692950053
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
pressure 53.83906896525934 LCOW --> 1.3736618890350945
----------------------------------------------------------------------

Degrees of Freedom: 


WaterTAP3 solution optimal 

----------------------------------------------------------------------
factor_membrane_replacement 0.3400000000000001 LCOW --> 1.3132086769188978
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
factor_membrane_replacement 0.3600000000000001 LCOW --> 1.3161330991946005
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
factor_membrane_replacement 0.38 LCOW --> 1.319057521469602
----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------
factor_membrane_replacement 0.4 LCOW --> 1.3219819437439706
--------------------------------------------

In [13]:
wt.run_water_tap(m=m, objective=True, print_model_results="summary", skip_small=True)

----------------------------------------------------------------------

Degrees of Freedom: 2

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


SW ONSHORE INTAKE:

	total cap investment: 2.00857
	cat and chem cost: 0.0
	electricity cost: 0.13389
	total fixed op cost: 0.02665

FERRIC CHLORIDE ADDITION:

	total cap investment: 1.6475
	cat and chem cost: 0.24895
	electricity cost: 0.0
	total fixed op cost: 0.02194

CHLORINATION:

	total cap investment: 5.20644
	cat and chem cost: 0.0345
	electricity cost: 1e-05
	total fixed op cost: 0.06933

STATIC MIXER:

	total cap investment: 0.11622
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.00154

HOLDING TANK:

	total cap investment: 1.48059
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.01946

MEDIA FILTRATION:

	total cap investment: 2.68667
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost

In [ ]:
# creates csv in results folder with the name: *case_study*_*scenario*.csv
# In this case, save the final baseline result.

df = wt.get_results_table(m=m, case_study=wt.case_study_trains.source_water["case_study"], 
                                scenario=wt.case_study_trains.source_water["scenario"])

In [66]:
#m.fs.objective_function.deactivate()

In [14]:
# lcow_list = []
# water_recovery_list = []
# scenario_value = []
# scenario_name = []
# elec_lcow = []
# elec_int = []

# runs_per_scenario = 20
# skip_small_sens = True
# # ############################################################    
# print("-------", "RESET", "-------")
# wt.run_water_tap(m = m, objective=True, skip_small = True)
# print("LCOW -->", m.fs.costing.LCOW())

# ############ salinity  +-30% ############
# stash_value = []
# for key in m.fs.flow_in_dict:   
#     stash_value.append(value(getattr(m.fs, key).conc_mass_in[0, "tds"]))
# scenario = "Inlet TDS +-20%"
# print("-------", scenario, "-------")
# ub = 1.15
# lb = 0.85
# step = (ub - lb) / runs_per_scenario

# for i in np.arange(lb, ub + step, step):
#     q=0
#     for key in m.fs.flow_in_dict:
#         getattr(m.fs, key).conc_mass_in[0, "tds"].fix(stash_value[q] * i)
#         q = q + 1

#     wt.run_water_tap(m = m, objective=True, skip_small = skip_small_sens)
#     print(scenario, stash_value[q-1] * i, "LCOW -->", m.fs.costing.LCOW())

#     lcow_list.append(value(m.fs.costing.LCOW))
#     water_recovery_list.append(value(m.fs.costing.system_recovery))
#     scenario_value.append(i)
#     scenario_name.append(scenario)
#     elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
#     elec_int.append(value(m.fs.costing.electricity_intensity))

# q=0
# for key in m.fs.flow_in_dict:    
#     getattr(m.fs, key).conc_mass_in[0, "tds"].fix(stash_value[q])
#     q = q + 1

In [15]:
# ############################################################
# print("-------", "RESET", "-------")
# # wt.run_water_tap(m = m, objective=False, skip_small = True)
# # print("LCOW -->", m.fs.costing.LCOW())
# ############ inlet flow +-30% ############
# stash_value = []
# for key in m.fs.flow_in_dict:   
#     stash_value.append(value(getattr(m.fs, key).flow_vol_in[0]))
# scenario = "Inlet Flow +-20%"
# print("-------", scenario, "-------")
# ub = 1.1
# lb = 0.9
# step = (ub - lb) / runs_per_scenario

# for i in np.arange(lb, ub + step, step):
#     q=0
#     for key in m.fs.flow_in_dict:
#         getattr(m.fs, key).flow_vol_in[0].fix(stash_value[q] * i)
#         q = q + 1

#     wt.run_water_tap(m = m, objective=False, skip_small = skip_small_sens)
#     print(scenario, stash_value[q-1] * i, "LCOW -->", m.fs.costing.LCOW())

#     lcow_list.append(value(m.fs.costing.LCOW))
#     water_recovery_list.append(value(m.fs.costing.system_recovery))
#     scenario_value.append(i)
#     scenario_name.append(scenario)
#     elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
#     elec_int.append(value(m.fs.costing.electricity_intensity))

# q=0
# for key in m.fs.flow_in_dict:    
#     getattr(m.fs, key).flow_vol_in[0].fix(stash_value[q])
#     q = q + 1